# Example 1 - Plot sample_rms for a target over time
The intent of this series of Jupyter Notebooks is to demonstrate how metrics can be retrieved from the sqlite metrics database and provide some ideas on how to use or plot those metrics.  

This example plots a timeseries of a single metric over time.

To run the example, it requires that there are sample_rms values
for 2020-10-01 through 2020-10-25 for IU.ANMO.00.BH1.M in a
database located at ./ispaq.db. To generate these values, you can run:

    python run_ispaq.py -M sample_rms -S ANMO --starttime 2020-10-01 --endtime 2020-10-26 --output db

This example will assume that the above command has already been run and the metrics already exist.


To begin, we need to import the necessary modules:

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime

Now we need to set some variables:

In [ ]:
db_name = '../ispaq.db'
metric = 'sample_rms'
startDate = '2020-10-01'
endDate = '2020-10-25'
target = 'IU.ANMO.00.BH1.M'
filename = f'example1_{target}_{startDate}_{endDate}.png'

The first step is to create a query that will be used to retrieve the sample_rms (or whatever metric you are using in the code block above).

In [ ]:
SQLcommand = f"SELECT * FROM {metric} WHERE start >= '{startDate}' " \
             f"and start < '{endDate }' and (target like '{target}');"

In [ ]:
print("\nCommand used to retrieve metrics from the sqlite database:")
print(SQLcommand)


Create a connection to the database and run the query, loading it into a pandas dataframe

In [ ]:
try:
   conn = sqlite3.connect(db_name)
   DF = pd.read_sql_query(SQLcommand, conn, parse_dates=['start','end'])
   conn.close
except Exception as e:
    print(f"Unable to connect to or find the {metric} table in the database {db_name}:\n{e}")

At this point, we have created a query to retrieve the metrics from the SQLite database, used sqlite3 to connect to the database, retreieved the metrics, closed the connection, and then ensured that the start times are in a datetime format for plotting purposes. 

This is what the dataframe looks like:

In [ ]:
print(DF)

For plotting purposes, we will create a new dataframe where each column (only one column in this case) is the metric and the associated values, and the index is the date of that value. 

In [ ]:
plotDF = pd.DataFrame()
plotDF[metric] = DF['value']
plotDF.index=DF['start']

In [ ]:
print(plotDF)

Now we use that dataframe to produce a plot.

In [ ]:
ax = plotDF.plot(style='.', color='k', title=metric)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
plt.minorticks_off()
date_form = DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)
plt.gcf().autofmt_xdate()

And save the plot for later viewing.

In [ ]:
plt.savefig(filename)